In [1]:
%load_ext autoreload
%autoreload 2
 # then imported files are u pdated each time a cell is executed (good if functions in py files are added and modified)

import scipy.io
import numpy as np
import plotly.graph_objects as go
import sys
import os

from pyswarms.single.global_best import GlobalBestPSO


os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from OsimPython.plotUtil import addArrow, addMarker
from Paper_util.IROS_2024_LMC.LMC_util import plotLMC, make_LMC,  check_LMC_Hand_visibility
from OsimPython.Hand_model import getHandModel, getHandMarkers, setHandPose, plot_hand, generate_hand_pose, generate_hand_poses

2024-02-17 11:56:31,463 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-02-17 11:56:31,464 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Make a Hand model
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)

In [3]:
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_poses(HandModel, state, num_poses=1, x_limits=[0, 0], y_limits=[0, 0], z_limits=[300, 300], limitsPS=[-70, 70], limis_FE=[-60, 60], limits_WD=[-60, 60])    
fig = go.Figure()
for pos_markers in handMarkers:
    fig = plot_hand(fig, pos_markers)
fig.show()

In [55]:
# Make a Hand model
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)

# Input parameter
fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                    [4,9,13,18], #index
                    [5,10,14,19], #middle
                    [6,11,15,20], #ring
                    [7,12,16,21]]) #pinky
    
finger_radius = 10 # mm
LMC_H = 600 # mm

num_LMC = 4
# Generate data
num_poses = 10
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_poses(HandModel, state, num_poses=num_poses, 
                                                                                       x_limits=[0, 0], y_limits=[0, 0], z_limits=[300, 300],
                                                                                         limitsPS=[-70, 70], limis_FE=[-60, 60], limits_WD=[-60, 60])
#handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_pose(HandModel, state, pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)

data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers]

# optimization
def fun(x, args):
    pos = x*mult
    cost = []
    data = args
    for xx in pos: # for each particle
        LMCs = [make_LMC(LMC_loc=np.array([xx[0+4*l], xx[1+4*l], 0]), LMC_orient=[xx[2+4*l], 0, xx[3+4*l]], LMC_H=LMC_H) for l in range(num_LMC)]
        occlusions = np.zeros([num_poses, num_LMC, 7])
        for lm, LMC in enumerate(LMCs):
            for i in range(num_poses):
                occlusions[i, lm] = check_LMC_Hand_visibility(LMC, pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], 
                                                              finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], 
                                                              palm_markers=data[1][i])

        occlusions = np.array(occlusions).astype(np.int16)
        #print(occlusions.shape) # num_poses, num_LMC, num
        vis = np.sum(occlusions, axis = 1)[:, 2:] # how many LMCs see each finger for each pose
        #print(vis.shape)
        vis = np.min(vis, axis=1) # take the minimum of each pose: e.g in pose i there is a finger that is only seen by 1 LMC -> this pose will get the value 1
        #print(vis)
        vis_LMC = np.zeros(fingers_idx.shape[0]) # counts how many poses have 1, 2, 3, 4, 5  fngers not seen byy any LMC
        unique, counts = np.unique(vis, return_counts=True)
        #print(vis_LMC, unique, counts)
        vis_LMC[unique] = counts

        met = 0
        '''
            this metrics functions in the following
            the number of poses having a finger that is not seen by and LMC is punished the most 
        '''
        # for f in range(fingers_idx.shape[0]-1): # for each finger:
        #     print(f, f+1)
        #     met += (vis_LMC[f] + 1) / (num_poses ** (f+1))
        # met /= (num_poses ** (fingers_idx.shape[0]-2)) 

        # maximize the global visibility of all fingers
        met += vis_LMC[0] / num_poses
        met += vis_LMC[1] / num_poses ** 2
        met += vis_LMC[2] / num_poses ** 3
        met += vis_LMC[3] / num_poses ** 4
        #met *= num_poses

        # maximize the local visibility of fingers in an LMC
        for lm, LMC in enumerate(LMCs):
            met_ = 0
            vis = np.sum(occlusions, axis = 1)[lm, 2:] # how many LMCs see each finger for each pose
            vis_LMC = np.zeros(fingers_idx.shape[0]) # counts how many poses have 1, 2, 3, 4, 5  fngers not seen byy any LMC
            unique, counts = np.unique(vis, return_counts=True)
            vis_LMC[unique] = counts

            met_ += vis_LMC[0] / num_poses
            met_ += vis_LMC[1] / num_poses ** 2
            met_ += vis_LMC[2] / num_poses ** 3
            met_ += vis_LMC[3] / num_poses ** 4
            #met_ *= num_poses ** 4
            #met += met_
            
            # penalize large LMC elevations 
            a_met = np.sum(np.abs(x[0, [2+4*lm, 3+4*lm]])) / (num_LMC*2*10) * met_
            met += a_met
            
            # maximize distance between LMCs (volume... to some extent)
            d_met = 0
            for lmm in range(num_LMC):
                if lmm == lm:
                    continue
                d_met += np.linalg.norm(LMCs[lm][0]-LMCs[lmm][0]) / (1000*num_LMC*100) * met_
            met -= d_met
            
        # penalize large LMC elevations 
        #a_met = np.sum(np.abs(x*np.array([0, 0, 1, 1]*num_LMC))) / (num_LMC*2*10)
        
        # # maximize distance between LMCs (volume... to some extent)
        # ds = 0
        # for lm in range(num_LMC):
        #     for lmm in range(lm, num_LMC):
        #         ds += np.linalg.norm(LMCs[lm][0]-LMCs[lmm][0]) / 1000
        # met -= ds * 10

        # try to maximize the distance from the LMCs to the center   
        # for lm, LMC in enumerate(LMCs):
        #     met -= np.linalg.norm(LMC[0]) / 1000 
        cost.append(met)
    return cost

mult = np.array([1000, 1000, 90, 180]*num_LMC)

# instatiate the optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.6}
x_max = np.array([1, 1, 1, 1]*num_LMC)
x_min = np.array([-1, -1, -1, -1]*num_LMC)
bounds = (x_min, x_max)
optimizer = GlobalBestPSO(n_particles=10, dimensions=4*num_LMC, options=options, bounds=bounds)

# now run the optimization, pass a=1 and b=100 as a tuple assigned to args
cost, pos = optimizer.optimize(fun, 100, args=(data))



2024-02-17 13:01:12,264 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.6}
pyswarms.single.global_best:   0%|          |0/100d:\Dokumente\Projects\HULD\Paper_util\IROS_2024_LMC\LMC_util.py:218: RuntimeWarning:

invalid value encountered in arccos

pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.0206
2024-02-17 13:10:05,808 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.020552181613429567, best pos: [ 0.15064258  0.14589982 -0.22812142  0.60887537 -0.16545953 -0.12516035
 -0.01322021 -0.01994919  0.31480264  0.22484185  0.25171776  0.03127642
 -0.06442255  0.38986011  0.23742635 -0.05655378]


In [61]:
def plotResult(pos, data_):
    deviceColor = ['red', 'blue', 'green', 'black']

    xx = pos*mult
    data = (data_)
    LMCs = [make_LMC(LMC_loc=[xx[0+4*l], xx[1+4*l], 0], LMC_orient=[xx[2+4*l], 0, xx[3+4*l]], LMC_H=LMC_H) for l in range(num_LMC)]
    occlusions = np.zeros([num_poses, num_LMC, 7])
    for lm, LMC in enumerate(LMCs):
        for i in range(num_poses):
            occlusions[i, lm] = check_LMC_Hand_visibility(LMC, pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i])

    bestLMCs = np.zeros(num_LMC)
    for i in range(num_poses):
        print('_________________________________')
        print('POS: ', i)
        for lm, LMC in enumerate(LMCs):
            print(occlusions[i, lm, 2:], np.sum(occlusions[i, lm, 2:]))
        k = np.sum(occlusions[i, :, 2:], axis = 1)
        kk = np.flatnonzero(k == np.max(k))
        bestLMCs[kk] += 1
        print('Best LMC: ', kk)

    print(bestLMCs)

    occlusions = np.array(occlusions).astype(np.int16)
    vis = np.sum(occlusions, axis = 1)[:, 2:] # how many LMCs see each finger for each pose
    print(vis)
    vis = np.min(vis, axis=1) # take the minimum of each pose: e.g in pose i there is a finger that is only seen by 1 LMC -> this pose will get the value 1
    vis_LMC = np.zeros(fingers_idx.shape[0]) # counts how many poses have 1, 2, 3, 4, 5  fngers not seen byy any LMC
    unique, counts = np.unique(vis, return_counts=True)
    vis_LMC[unique] = counts
    
    fig = go.Figure()
    for lm, LMC in enumerate(LMCs):
        fig = plotLMC(fig, LMC, color=deviceColor[lm], name='LMC_{}'.format(lm), scale = 1)

    #fig = go.Figure()
    for pos_markers in handMarkers:
        fig = plot_hand(fig, pos_markers)
    #fig = plot_hand(fig, data[0][0])
    fig.show()

#print(pos)
plotResult(pos, data)

d:\Dokumente\Projects\HULD\Paper_util\IROS_2024_LMC\LMC_util.py:233: RuntimeWarning:

invalid value encountered in arccos



_________________________________
POS:  0
[0. 0. 0. 0. 0.] 0.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
Best LMC:  [1 2 3]
_________________________________
POS:  1
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
Best LMC:  [0 1 2 3]
_________________________________
POS:  2
[0. 0. 0. 0. 0.] 0.0
[1. 0. 1. 1. 0.] 3.0
[1. 0. 1. 1. 0.] 3.0
[1. 1. 0. 0. 1.] 3.0
Best LMC:  [1 2 3]
_________________________________
POS:  3
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
Best LMC:  [0 1 2 3]
_________________________________
POS:  4
[0. 0. 0. 0. 0.] 0.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
Best LMC:  [1 2 3]
_________________________________
POS:  5
[1. 1. 1. 1. 1.] 5.0
[1. 1. 1. 1. 1.] 5.0
[0. 0. 0. 0. 0.] 0.0
[1. 1. 1. 1. 1.] 5.0
Best LMC:  [0 1 3]
_________________________________
POS:  6
[0. 0. 0. 0. 0.] 0.0
[1. 1. 1. 0. 0.] 3.0
[1. 1. 1. 0. 1.] 4.0
[1. 1. 1. 1. 0.] 4.0


In [76]:
x = pos * mult
print(x)
#x = np.array([300, 0, 0, 0])
LMCs = [make_LMC(LMC_loc=[x[0+4*l], x[1+4*l], 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) for l in range(num_LMC)]

occlusions = []
for LMC in LMCs:
    for i in range(num_poses):
        occlusions += [check_LMC_Hand_visibility(LMC, pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i])]
        

occlusions = np.array(occlusions)
print(occlusions)
fig = go.Figure()
fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, data[0][0])
fig.show()

[ 3.71477453e+02 -4.97280161e+01  9.57858303e-05  6.87211000e+01]
[[1. 1. 1. 1. 1. 1. 1.]]


In [ ]:
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers = generate_hand_pose(pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)
data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers]
LMCs = [make_LMC(LMC_loc=[-380,  130, 0], LMC_orient=[0, 0, 0], LMC_H=LMC_H) ]
i = 0
print(check_LMC_Hand_visibility(LMCs[0], pos_markers=data[0][i], fingers_idx=fingers_idx, finger_lines=data[4][0], finger_radius=finger_radius, palm_plane_normal=data[2][i], palm_centroid=data[3][i], palm_markers=data[1][i]))

fig = go.Figure()
fig = plotLMC(fig, LMCs[0], color='orange', name='LMC', scale = 30)
fig = plot_hand(fig, data[0][0])
fig.show()

In [ ]:
d = np.zeros(5)
k = np.ones(5)
for i in range(4):
    k[i+1] = 2*k[-1]
    d[i+1] = 